In [72]:
import math
import numpy as np
import pandas as pd

In [78]:
# Training and test data files
train_file = '../input/train.csv'
test_file = '../input/test.csv'
model_file = '../output/titanic.model.json'
model_weights_file = '../output/titanic.model.best.hdf5'
pred_file = '../output/gender_submission.csv'

In [79]:
# Load training data
df_train = pd.read_csv(train_file)
print(df_train.shape)
df_train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [80]:
# Extract survived data as predictions: 0 = Died, 1 = Survived
from keras.utils.np_utils import to_categorical
y_train = to_categorical(df_train["Survived"], 2)
df_train.pop('Survived')
print(y_train.shape)
print(y_train[0:5])

(891, 2)
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [81]:
# Prepare the rest of the data for training
from sklearn.preprocessing import MinMaxScaler

max_name_len = df_train.Name.map(len).max()    

def prep_data(frame):
    frame = frame.fillna(0)

    # Creating new family_size and fare per person columns 
    frame['Family_Size'] = frame['SibSp'] + frame['Parch'] + 1
    frame['Fare_Per_Person'] = frame['Fare']/frame['Family_Size']

    # Convert Sex and Embarked to number
    frame['Sex'] = pd.Categorical(frame['Sex']).codes
    frame['Embarked'] = pd.Categorical(frame['Embarked']).codes
    
    # Convert name
    for i in range(0, max_name_len):
        col_name = 'Name' + str(i)
        frame[col_name] = frame['Name'].str[i]
        frame[col_name] = frame.apply(lambda row: ord(' ') if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Name')
    
    # TODO: Ignore Ticket, Cabin for now
    frame.pop('Ticket')
    frame.pop('Cabin')
    frame.pop('PassengerId')
    
    min_max_scaler = MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(frame)
    return pd.DataFrame(np_scaled)


In [82]:
# Prepare the rest of the data for training
df_train = prep_data(df_train)
X_train = np.array(df_train)[:,:]
X_train = X_train.astype('float32')
print(X_train.shape)
print(X_train[0])

(891, 91)
[ 1.          1.          0.27500001  0.125       0.          0.01415106
  1.          0.1         0.00707553  0.01886792  0.91463417  0.72222221
  0.94444442  0.86666667  0.75555557  0.13483146  0.          0.50561798
  0.95348835  0.15730338  0.          0.54022986  0.9775281   0.76666665
  0.87640452  0.          0.44444445  0.72222221  0.91111112  0.91111112
  0.81111109  0.93258429  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.     

In [83]:
# Build a training network

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1], )))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 512)               47104     
_________________________________________________________________
dropout_25 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_26 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_27 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                8256      
__________

In [84]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=model_weights_file, verbose=1, save_best_only=True)
#stopper = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, mode='auto')
hist = model.fit(X_train, y_train, epochs=1000, batch_size=100, validation_split=0.2, callbacks=[checkpointer], verbose=1, shuffle=True)

Train on 712 samples, validate on 179 samples
Epoch 1/1000
712/712 [==============================] - 0s - loss: 0.6982 - acc: 0.5702 - val_loss: 0.6683 - val_acc: 0.7039
Epoch 2/1000
712/712 [==============================] - 0s - loss: 0.6718 - acc: 0.6067 - val_loss: 0.6249 - val_acc: 0.7151
Epoch 3/1000
712/712 [==============================] - 0s - loss: 0.6421 - acc: 0.6236 - val_loss: 0.5925 - val_acc: 0.7486
Epoch 4/1000
712/712 [==============================] - 0s - loss: 0.6236 - acc: 0.6685 - val_loss: 0.5640 - val_acc: 0.7877
Epoch 5/1000
712/712 [==============================] - 0s - loss: 0.6012 - acc: 0.6756 - val_loss: 0.5366 - val_acc: 0.7821
Epoch 6/1000
712/712 [==============================] - 0s - loss: 0.5774 - acc: 0.7022 - val_loss: 0.5113 - val_acc: 0.7765
Epoch 7/1000
712/712 [==============================] - 0s - loss: 0.5803 - acc: 0.7037 - val_loss: 0.5173 - val_acc: 0.7486
Epoch 8/1000
712/712 [==============================] - 0s - loss: 0.5579 - acc

712/712 [==============================] - 0s - loss: 0.4441 - acc: 0.8104 - val_loss: 0.4061 - val_acc: 0.8268
Epoch 29/1000
712/712 [==============================] - 0s - loss: 0.4309 - acc: 0.8230 - val_loss: 0.4065 - val_acc: 0.8268
Epoch 30/1000
712/712 [==============================] - 0s - loss: 0.4361 - acc: 0.8160 - val_loss: 0.4140 - val_acc: 0.8268
Epoch 31/1000
712/712 [==============================] - 0s - loss: 0.4368 - acc: 0.8160 - val_loss: 0.4126 - val_acc: 0.8156
Epoch 32/1000
712/712 [==============================] - 0s - loss: 0.4079 - acc: 0.8287 - val_loss: 0.4099 - val_acc: 0.8268
Epoch 33/1000
712/712 [==============================] - 0s - loss: 0.4184 - acc: 0.8287 - val_loss: 0.4042 - val_acc: 0.8156
Epoch 34/1000
712/712 [==============================] - 0s - loss: 0.4102 - acc: 0.8385 - val_loss: 0.4021 - val_acc: 0.8101
Epoch 35/1000
712/712 [==============================] - 0s - loss: 0.4102 - acc: 0.8427 - val_loss: 0.4090 - val_acc: 0.8156
Epoch 

712/712 [==============================] - 0s - loss: 0.3096 - acc: 0.8722 - val_loss: 0.4692 - val_acc: 0.8212
Epoch 62/1000
712/712 [==============================] - 0s - loss: 0.3194 - acc: 0.8750 - val_loss: 0.4785 - val_acc: 0.8156
Epoch 63/1000
712/712 [==============================] - 0s - loss: 0.2933 - acc: 0.8862 - val_loss: 0.4864 - val_acc: 0.7989
Epoch 64/1000
712/712 [==============================] - 0s - loss: 0.3253 - acc: 0.8806 - val_loss: 0.4940 - val_acc: 0.8268
Epoch 65/1000
712/712 [==============================] - 0s - loss: 0.3036 - acc: 0.8820 - val_loss: 0.4860 - val_acc: 0.8101
Epoch 66/1000
712/712 [==============================] - 0s - loss: 0.3058 - acc: 0.8806 - val_loss: 0.5041 - val_acc: 0.7821
Epoch 67/1000
712/712 [==============================] - 0s - loss: 0.2816 - acc: 0.8933 - val_loss: 0.5044 - val_acc: 0.8045
Epoch 68/1000
712/712 [==============================] - 0s - loss: 0.2818 - acc: 0.8862 - val_loss: 0.5281 - val_acc: 0.8045
Epoch 

712/712 [==============================] - 0s - loss: 0.2293 - acc: 0.9129 - val_loss: 0.5913 - val_acc: 0.7989
Epoch 96/1000
712/712 [==============================] - 0s - loss: 0.2345 - acc: 0.9185 - val_loss: 0.5537 - val_acc: 0.7989
Epoch 97/1000
712/712 [==============================] - 0s - loss: 0.2290 - acc: 0.9017 - val_loss: 0.6128 - val_acc: 0.7877
Epoch 98/1000
712/712 [==============================] - 0s - loss: 0.2389 - acc: 0.9101 - val_loss: 0.6045 - val_acc: 0.8045
Epoch 99/1000
712/712 [==============================] - 0s - loss: 0.2544 - acc: 0.9059 - val_loss: 0.5652 - val_acc: 0.7989
Epoch 100/1000
712/712 [==============================] - 0s - loss: 0.2255 - acc: 0.9115 - val_loss: 0.5962 - val_acc: 0.7933
Epoch 101/1000
712/712 [==============================] - 0s - loss: 0.2142 - acc: 0.9157 - val_loss: 0.6329 - val_acc: 0.7989
Epoch 102/1000
712/712 [==============================] - 0s - loss: 0.2112 - acc: 0.9171 - val_loss: 0.6301 - val_acc: 0.7989
Epo

712/712 [==============================] - 0s - loss: 0.2068 - acc: 0.9242 - val_loss: 0.7052 - val_acc: 0.7821
Epoch 129/1000
712/712 [==============================] - 0s - loss: 0.2074 - acc: 0.9256 - val_loss: 0.6973 - val_acc: 0.7709
Epoch 130/1000
712/712 [==============================] - 0s - loss: 0.2171 - acc: 0.9045 - val_loss: 0.6836 - val_acc: 0.7933
Epoch 131/1000
712/712 [==============================] - 0s - loss: 0.1956 - acc: 0.9157 - val_loss: 0.6905 - val_acc: 0.7877
Epoch 132/1000
712/712 [==============================] - 0s - loss: 0.1815 - acc: 0.9157 - val_loss: 0.7322 - val_acc: 0.7933
Epoch 133/1000
712/712 [==============================] - 0s - loss: 0.1842 - acc: 0.9270 - val_loss: 0.7622 - val_acc: 0.7709
Epoch 134/1000
712/712 [==============================] - 0s - loss: 0.1712 - acc: 0.9270 - val_loss: 0.7815 - val_acc: 0.7654
Epoch 135/1000
712/712 [==============================] - 0s - loss: 0.1562 - acc: 0.9368 - val_loss: 0.7926 - val_acc: 0.7933

...

712/712 [==============================] - 0s - loss: 0.0436 - acc: 0.9803 - val_loss: 1.7834 - val_acc: 0.7989
Epoch 866/1000
712/712 [==============================] - 0s - loss: 0.0693 - acc: 0.9691 - val_loss: 1.7760 - val_acc: 0.8045
Epoch 867/1000
712/712 [==============================] - 0s - loss: 0.0667 - acc: 0.9635 - val_loss: 1.7379 - val_acc: 0.7989
Epoch 868/1000
712/712 [==============================] - 0s - loss: 0.0625 - acc: 0.9705 - val_loss: 1.8281 - val_acc: 0.7933
Epoch 869/1000
712/712 [==============================] - 0s - loss: 0.0465 - acc: 0.9775 - val_loss: 1.8602 - val_acc: 0.7821
Epoch 870/1000
712/712 [==============================] - 0s - loss: 0.0516 - acc: 0.9747 - val_loss: 1.8391 - val_acc: 0.7877
Epoch 871/1000
712/712 [==============================] - 0s - loss: 0.0576 - acc: 0.9789 - val_loss: 1.8779 - val_acc: 0.7877
Epoch 872/1000
712/712 [==============================] - 0s - loss: 0.0678 - acc: 0.9691 - val_loss: 1.8944 - val_acc: 0.7598

712/712 [==============================] - 0s - loss: 0.0574 - acc: 0.9649 - val_loss: 1.9774 - val_acc: 0.7933
Epoch 900/1000
712/712 [==============================] - 0s - loss: 0.0606 - acc: 0.9677 - val_loss: 2.0003 - val_acc: 0.7877
Epoch 901/1000
712/712 [==============================] - 0s - loss: 0.0595 - acc: 0.9775 - val_loss: 2.0803 - val_acc: 0.7877
Epoch 902/1000
712/712 [==============================] - 0s - loss: 0.0566 - acc: 0.9733 - val_loss: 2.0175 - val_acc: 0.7877
Epoch 903/1000
712/712 [==============================] - 0s - loss: 0.0462 - acc: 0.9775 - val_loss: 1.9637 - val_acc: 0.7821
Epoch 904/1000
712/712 [==============================] - 0s - loss: 0.0809 - acc: 0.9761 - val_loss: 1.9300 - val_acc: 0.7877
Epoch 905/1000
712/712 [==============================] - 0s - loss: 0.0660 - acc: 0.9621 - val_loss: 1.9645 - val_acc: 0.7933
Epoch 906/1000
712/712 [==============================] - 0s - loss: 0.0466 - acc: 0.9719 - val_loss: 2.0369 - val_acc: 0.7933

712/712 [==============================] - 0s - loss: 0.0488 - acc: 0.9761 - val_loss: 2.0201 - val_acc: 0.7877
Epoch 933/1000
712/712 [==============================] - 0s - loss: 0.0559 - acc: 0.9719 - val_loss: 1.9910 - val_acc: 0.7821
Epoch 934/1000
712/712 [==============================] - 0s - loss: 0.0573 - acc: 0.9761 - val_loss: 2.0122 - val_acc: 0.7765
Epoch 935/1000
712/712 [==============================] - 0s - loss: 0.0481 - acc: 0.9747 - val_loss: 2.0432 - val_acc: 0.7765
Epoch 936/1000
712/712 [==============================] - 0s - loss: 0.0510 - acc: 0.9733 - val_loss: 2.0734 - val_acc: 0.7765
Epoch 937/1000
712/712 [==============================] - 0s - loss: 0.0578 - acc: 0.9733 - val_loss: 2.0844 - val_acc: 0.7765
Epoch 938/1000
712/712 [==============================] - 0s - loss: 0.0637 - acc: 0.9663 - val_loss: 2.1126 - val_acc: 0.7765
Epoch 939/1000
712/712 [==============================] - 0s - loss: 0.0510 - acc: 0.9803 - val_loss: 2.0752 - val_acc: 0.7933

712/712 [==============================] - 0s - loss: 0.0520 - acc: 0.9789 - val_loss: 2.1074 - val_acc: 0.7821
Epoch 967/1000
712/712 [==============================] - 0s - loss: 0.0460 - acc: 0.9775 - val_loss: 2.1008 - val_acc: 0.7821
Epoch 968/1000
712/712 [==============================] - 0s - loss: 0.0354 - acc: 0.9803 - val_loss: 2.1013 - val_acc: 0.7877
Epoch 969/1000
712/712 [==============================] - 0s - loss: 0.0536 - acc: 0.9733 - val_loss: 2.1075 - val_acc: 0.7877
Epoch 970/1000
712/712 [==============================] - 0s - loss: 0.0424 - acc: 0.9775 - val_loss: 2.0527 - val_acc: 0.7989
Epoch 971/1000
712/712 [==============================] - 0s - loss: 0.0588 - acc: 0.9719 - val_loss: 2.0759 - val_acc: 0.8045
Epoch 972/1000
712/712 [==============================] - 0s - loss: 0.0459 - acc: 0.9733 - val_loss: 2.1463 - val_acc: 0.8045
Epoch 973/1000
712/712 [==============================] - 0s - loss: 0.0457 - acc: 0.9789 - val_loss: 1.9966 - val_acc: 0.7989

712/712 [==============================] - 0s - loss: 0.0648 - acc: 0.9719 - val_loss: 2.0634 - val_acc: 0.8045
Epoch 1000/1000
712/712 [==============================] - 0s - loss: 0.0518 - acc: 0.9747 - val_loss: 1.8379 - val_acc: 0.7709


In [85]:
# Evaluating the model on the training set
score = model.evaluate(X_train, y_train)
print("\n Training Accuracy:", score[1])

 32/891 [>.............................] - ETA: 0s
 Training Accuracy: 0.950617284151


In [90]:
# Load test data
df_test = pd.read_csv(test_file)
print(df_test.shape)
df_test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [91]:
# Prepare the data for testing
start_passenger_id = df_test.iloc[0]['PassengerId']
print(start_passenger_id)
df_test = prep_data(df_test)
X_test = np.array(df_test)[:,:]
X_test = X_test.astype('float32')
print(X_test.shape)
print(X_test[0])

892
(418, 91)
[ 1.          1.          0.45394737  0.          0.          0.01528158
  0.5         0.          0.02983973  0.18867925  0.75609756  0.85393256
  0.84444445  0.98888886  0.13333334  0.          0.5         0.94252872
  0.15555556  0.          0.49411765  0.74712646  0.89534885  0.77528089
  0.9222222   0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
 

In [93]:
# Predict for test data
y_test = model.predict(X_test)
print(y_test[0])

[  7.04982886e-05   9.99985695e-01]


In [95]:
# Save predictions
with open(pred_file, 'w') as f:
    f.write('PassengerId,Survived\n')
    for index, y_hat in enumerate(y_test):
        prediction = np.argmax(y_hat)
        f.write(str(start_passenger_id)+','+str(prediction)+'\n')
        start_passenger_id = start_passenger_id + 1
    f.close()